In [ ]:
import nltk
nltk.download()
# here to download movie_reviews and stopwords packages

In [ ]:
from nltk.corpus import movie_reviews as mr
print(mr.readme())

from nltk.corpus import stopwords
#print(stopwords.words('english'))
sw = stopwords.words('english');
print(len(sw))


# here in the stopwords list 'sw'
# it should remove some words from the list
# such as: 'no','nor','not','too',
sw.remove('no')
sw.remove('nor')
sw.remove('not')
sw.remove('too')
# "don't","aren't", "couldn't", 'didn', "didn't", 
# "doesn't", "hadn't", "hasn't", "haven't", "isn't", "mightn't",
#  "mustn't", "needn't", "shan't", "shouldn't", "wasn't", "weren't",
#  "won't", "wouldn't"
sw1 = []
for v in sw:
    if not v.endswith("n't"):
        sw1.append(v)
    #else:
        #print(v)
sw1.remove('ain')
sw1.remove('aren')
sw1.remove('couldn')
sw1.remove('didn')
sw1.remove('doesn')
sw1.remove('hadn')
sw1.remove('haven')
sw1.remove('isn')
sw1.remove('mightn')
sw1.remove('hasn')
sw1.remove('mustn')
sw1.remove('needn')
sw1.remove('shan')
sw1.remove('wasn')
sw1.remove('weren')
sw1.remove('wouldn')
sw = sw1
sw

In [ ]:
# divide the movie_reviews into 2 categories: positive and negative
positive = mr.fileids('pos')
negative = mr.fileids('neg')

# form a new movie_reviews as a dictionary by positive and negative
new_mr = dict(pos = positive, neg = negative)

# the length of positive and negative lists are both 1000
posLen = len(new_mr['pos'])
print(posLen)
negLen = len(new_mr['neg'])
print(negLen)

# get by text string
#print(mr.raw(new_mr['pos'][0]))
print(len(mr.raw(new_mr['pos'][0])))
#print(mr.raw(new_mr['neg'][0]))
print(len(mr.raw(new_mr['neg'][0])))
# get by words
#print(mr.words(new_mr['pos'][0])
print(len(mr.words(new_mr['pos'][0])))
#print(mr.words(new_mr['pos'][0])
print(len(mr.words(new_mr['neg'][0])))



1000
1000
4227
4043
862
879


In [ ]:
# the input is the whole text string
# improvement: remove stopwords and other useless words, punctuations
def unigram_features (words):
    """
    This is the simplest possible feature representation of a document.

    Each word is a feature.
    """
    return dict((word, True) for word in words)

# input a word, 
# determine whether it is a stopword or not
# remove all punctuations from the word
def pre_process(word):
  # remove punctuations except apostrophe
  w = ""
  hasAlphabet = False
  for i in word:
    if i >= 'a' and i <= 'z':
      w = w + i
      hasAlphabet = True
    elif i >= 'A' and i <= 'Z':
      w = w + i
      hasAlphabet = True
    elif i == '\'': # apostrophe
      w = w + i
  #print(w)
  # check whether it is stopword
  if hasAlphabet and w not in sw:
    return w
  return ''

word = ",,wouldn't.."
pre_process(word);


# the input parameter is a list consisting of words
# and return a dictionary which includes useful words after pre-processing
def feature_selection(words):
  wd = {}
  for word in words:
    word = pre_process(word)
    if len(word) > 0 and word not in wd:
      wd[word] = True
  return wd

d1 = feature_selection(mr.words(new_mr['pos'][0]))
print(len(d1))
print(d1)
d2 = unigram_features(mr.words(new_mr['pos'][0]))
print(len(d2))
print(d2)

d1 = feature_selection(mr.words(new_mr['neg'][0]))
print(len(d1))
print(d1)
d2 = unigram_features(mr.words(new_mr['neg'][0]))
print(len(d2))
print(d2)

# extract a file that is a move review from the nltk.corpus's movie_reviews
def extract_features (corpus, file_ids, cls, feature_extractor=unigram_features):
    """
    Turn a set of files all belonging to one class into a list
    of (feature dictionary, cls) pairs, to be used in testing or training
    a classifier.
    """
    return [(feature_extractor(corpus.words(i)), cls) for i in file_ids]

# collect several/some/many dict-type positive or negative movie reviews into a list
# for training or testing
# the returned list is composed of tuple
# every tuple is the ict-type positive or negative movie reviews + clf
# corpus, should be the 'mr'
# files, should be the 'new_mr'
# clf, is the classification: pos or neg
def collect_features(corpus, files, startID, endID, clf):
  col = []
  i = startID
  while i < endID:
    dic = feature_selection(corpus.words(files[clf][i]))
    tup = (dic, clf)
    col.append(tup)
    i = i + 1
  return col

# return every word from the given removie reviews
# file_ids, means the the range, to get all movie reviews in this range
def get_words_from_corpus (corpus, file_ids):
    for file_id in file_ids:
        words = corpus.words(file_id)
        for word in words:
            yield word


tmp = extract_features(mr, new_mr['neg'][:100], 'neg', feature_extractor=feature_selection)
print(len(tmp))
print(tmp[0])
print(len(tmp[0]))
print()

# for example, choose the first 100 files to be the training set
tmp = collect_features(mr, new_mr, 0, 100, 'neg')
print(len(tmp))
print(tmp[0])
print(len(tmp[0]))
print()

In [ ]:
# selection the 3000 most important words.
# find the 3000 most important words by frequency
# top, is how many words in the list to be returned
def importantWords(corpus, file, top):
  iw = []
  words_dic = {}
  pLen = len(file['pos'])
  i = 0
  while i < pLen:
    movie_review = corpus.words(file['pos'][i])
    for w in movie_review:
      w = pre_process(w)
      if w not in words_dic:
        words_dic[w] = 1
      else:
        words_dic[w] = words_dic[w] + 1
    i = i + 1
  nLen = len(file['neg'])
  i = 0
  while i < nLen:
    movie_review = corpus.words(file['neg'][i])
    for w in movie_review:
      w = pre_process(w)
      if len(w) > 0:
        if w not in words_dic:
          words_dic[w] = 1
        else:
          words_dic[w] = words_dic[w] + 1
      else:
        continue
    i = i + 1
  words_dic.pop('') # remove space
  sorted_words_dic = sorted(words_dic.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
  # add the top words to the list to be returned
  i = 0
  while i < top:
    iw.append(sorted_words_dic[i])
    i = i + 1
  return iw

top = 10
importantWords(mr, new_mr, top)

[('film', 9519),
 ('one', 5853),
 ('movie', 5774),
 ('not', 5583),
 ('like', 3690),
 ('even', 2565),
 ('no', 2473),
 ('time', 2411),
 ('good', 2411),
 ('story', 2170)]

In [ ]:
# Use a Naive Bayes Classifier
#from nltk.classify import NaiveBayesClassifier
#data = dict(pos = mr.fileids('pos'),neg = mr.fileids('neg'))

# Use 90% of the data for training
test_start_index = 900
neg_training = extract_features(mr, data['neg'][:test_start_index], 'neg', feature_extractor=unigram_features)
#print(neg_training[0])
# Use 10% for testing the classifier on unseen data.
neg_test = extract_features(mr, data['neg'][test_start_index:], 'neg', feature_extractor=unigram_features)

pos_training = extract_features(mr, data['pos'][:test_start_index],'pos', feature_extractor=unigram_features)
pos_test = extract_features(mr, data['pos'][test_start_index:],'pos', feature_extractor=unigram_features)

train_set = pos_training + neg_training
test_set = pos_test + neg_test

print(len(neg_training))
print(len(neg_test))
print(len(pos_training))
print(len(pos_test))
print(len(train_set))
print(len(test_set))
print()

# 
startID = 0
testID = 900
pos_training = collect_features(mr, new_mr, startID, testID, 'pos')
pos_test = collect_features(mr, new_mr, testID, posLen, 'pos')

neg_training = collect_features(mr, new_mr, startID, testID, 'neg')
neg_test = collect_features(mr, new_mr, testID, negLen, 'neg')

# whole sets
train_set = pos_training + neg_training
test_set = pos_test + neg_test

print(len(neg_training))
print(len(neg_test))
print(len(pos_training))
print(len(pos_test))
print(len(train_set))
print(len(test_set))

In [ ]:
# text classification
# Use a Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier
# Train a classifier on our training data.
classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
def get_review_text (clf,file_id,start=0,end=None):
    words = list(mr.words(data[clf][file_id]))
    return ' '.join(words[start:end])

def get_mr_text(corpus, files, clf, fileID, startID=0, endID=None):
  words = list(corpus.words(files[clf][fileID]))
  return ' '.join(words[startID:endID])

In [ ]:
#print(get_mr_text(mr, new_mr, 'pos', 900) )
#print(get_mr_text(mr, new_mr, 'neg', 900) )
pre_pos_right = 0
pre_neg_right = 0
pre_pos_wrong = 0
pre_neg_wrong = 0
i = 0
while i < 100:
  predicted_label0 = classifier.classify(pos_test[i][0])
  #print(pos_test[i][0])
  if predicted_label0 == 'pos':
    pre_pos_right = pre_pos_right + 1
  if predicted_label0 == 'neg':
    pre_pos_wrong = pre_pos_wrong + 1
  print('Predicted: %s \t Actual: pos ' %predicted_label0)
  predicted_label1 = classifier.classify(neg_test[i][0])
  #print(neg_test[i][0])
  if predicted_label1 == 'neg':
    pre_neg_right = pre_neg_right + 1
  if predicted_label1 == 'pos':
    pre_neg_wrong = pre_neg_wrong + 1
  print('Predicted: %s \t Actual: neg ' %predicted_label1)
  i = i + 1

print(pre_pos_right)
print(pre_neg_right)
print(pre_pos_wrong)
print(pre_neg_wrong)

In [ ]:
# new testing examples excluded from the move_reviews set
prediction = classifier.classify(unigram_features('Inception is the best movie ever'.split()))
print(prediction)
prediction = classifier.classify(unigram_features("I don't know how anyone could sit through Inception".split()))
print(prediction)

classifier.show_most_informative_features()

# 
prediction = classifier.classify(feature_selection('Inception is the best movie ever'.split()))
print(prediction)
prediction = classifier.classify(feature_selection("I don't know how anyone could sit through Inception".split()))
print(prediction)

classifier.show_most_informative_features()

In [ ]:
# Serious testing
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def do_evaluation (pairs, pos_label='pos', verbose=True):
    predicted, actual = zip(*pairs)
    precision = precision_score(actual,predicted,pos_label=pos_label)
    recall = recall_score(actual,predicted,pos_label=pos_label)
    accuracy = accuracy_score(actual,predicted)
    f1 = f1_score(actual,predicted,pos_label=pos_label)
    if verbose:
      print_results(precision, recall, accuracy, f1, pos_label)
    return (precision, recall, accuracy, f1)

def print_results (precision, recall, accuracy, f1, pos_label):
    banner =  'Evaluation with pos label = %s' % pos_label
    print()
    print(banner)
    print('=' * len(banner))
    print('{0:10s} {1:.1f} %'.format('Precision',precision*100))
    print('{0:10s} {1:.1f} %'.format('Recall',recall*100))
    print('{0:10s} {1:.1f} %'.format('Accuracy',accuracy*100))
    print('{0:10s} {1:.1f} %'.format('F1 score',f1*100))


In [ ]:
pairs = [(classifier.classify(example), actual) for (example, actual) in test_set]
print(pairs)
print(pairs[0])
print(pairs[99])
print(pairs[100])
print(pairs[199])
print()
i = 0
while i < 100:
  if pairs[i][0] == 'neg':
    print(pairs[i])
  i = i + 1

# 
precision, recall, accuracy, f1 = do_evaluation (pairs, pos_label='pos')
precision, recall, accuracy, f1 = do_evaluation (pairs, pos_label='neg')

[('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('neg', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('neg', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos', 'pos'), ('pos',

In [ ]:
pairs = [(classifier.classify(example), actual) for (example, actual) in test_set]
#pairs = [(classifier.classify(example), actual) for (example, actual) in test_set]

print(len(pairs[0]))
print( pairs[0] )


precision, recall, accuracy = do_evaluation (pairs, pos_label='pos')
print(precision)
print(recall)
print(accuracy)
precision, recall, accuracy = do_evaluation (pairs, pos_label='neg')
print(precision)
print(recall)
print(accuracy)

pos_guesses = [p for (p,a) in pairs if p=='pos']
pos_actual = [a for (p,a) in pairs if a=='pos']
do_evaluation (pairs, pos_label='neg')
print('Note that {:.1%} of our classifier guesses were positive'.format(float(len(pos_guesses))/len(pairs)))
print('While {:.1%} of the reviews were actually positive'.format(float(len(pos_actual))/len(pairs)))
# to see the actual pairs that came out of the test uncomment the next line
#pairs

2
('pos', 'pos')
pos

Evaluation with pos label = pos
Precision  64.7
Recall     97.0
Accuracy   72.0
0.6466666666666666
0.97
0.72

Evaluation with pos label = neg
Precision  94.0
Recall     47.0
Accuracy   72.0
0.94
0.47
0.72

Evaluation with pos label = neg
Precision  94.0
Recall     47.0
Accuracy   72.0
Note that 75.0% of our classifier guesses were positive
While 50.0% of the reviews were actually positive


In [ ]:
# SVM Classifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

import os.path

def add_data_from_files (file_list,data_list):
    for f in file_list:
        with open(f,'r') as fh:
            data_list.append(fh.read())

home = os.getenv('HOME')
# This is where MY NLTK data is.  Yours should be in a similar place relative
# to what your machine thinks is HOME.
data_dir = os.path.join(home,'nltk_data/corpora/movie_reviews/')

clses = ['pos','neg']

#  The data is in the data_dir, sorted into subdirectories, one for each class.
data_dirs = [os.path.join(data_dir,cls) for cls in clses]
#  We use a somewhat more traditional feature weights, called TFIDF weights
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

# We're going to compute 4 lists training data and labels, test data a nd labels
train_labels = []
test_labels = []

train_data = []
test_data = []
training_proportion = (9,10)
for i,cls  in enumerate(clses):
    d_dir = data_dirs[i]
    os.chdir(d_dir)
    cls_files = os.listdir(d_dir)
    num_cls_files = len(cls_files)
    training_index = int(training_proportion[0] *(num_cls_files/training_proportion[1]))
    train_labels.extend(cls for f in cls_files[:training_index])
    test_labels.extend(cls for f in cls_files[training_index:])
    add_data_from_files (cls_files[:training_index],train_data)
    add_data_from_files (cls_files[training_index:],test_data)

# Now with data set represented as a list of strings (one from each file),
# extract the TFIDF features
train_features = vectorizer.fit_transform(train_data)

#  We extract features from the test data using the same vectorizer
#  trained on training data. The TFIDF feature model has been fit to
#  (depends only on) the training data.
test_features = vectorizer.transform(test_data)

# Create an SVM classifier instance
clf = LinearSVC(loss='squared_hinge', penalty="l2", dual=False, tol=1e-3)

# Train (or "fit") the model to the training data.
clf.fit(train_features, train_labels)

# Test the model on the test data.
predicted_labels = clf.predict(test_features)

# Evaluate the results
pos_guesses = [p for p in predicted_labels if p=='pos']
pos_actual = [p for p in test_labels if p=='pos']
print('Note that {:.1%} of our classifier guesses were positive'.format(float(len(pos_guesses))/len(test_labels)))
print('While {:.1%} of the reviews were actually positive'.format(float(len(pos_actual))/len(test_labels)))
en = do_evaluation (zip(predicted_labels,test_labels), pos_label='pos', verbose=True)
print(en)
en = do_evaluation (zip(predicted_labels,test_labels), pos_label= 'neg', verbose=True)
print(en)

Note that 52.0% of our classifier guesses were positive
While 50.0% of the reviews were actually positive

Evaluation with pos label = pos
Precision  84.6
Recall     88.0
Accuracy   86.0
(0.8461538461538461, 0.88, 0.86)

Evaluation with pos label = neg
Precision  87.5
Recall     84.0
Accuracy   86.0
(0.875, 0.84, 0.86)
